## 控制迷宫寻宝机器人

在这个项目中，你将使用刚刚学到的知识，尝试根据要求，编写代码，来控制一个机器人，在模拟环境中行走，并找到目标宝藏。

机器人所在的模拟环境中，会包含这样几个因素：机器人的起点、障碍物、宝藏箱。你的任务包括：

1. 分析模拟环境的数据
2. 控制机器人随机行动
3. （可选）控制机器人走到终点


* 一个良好的含有注释的代码，可以让你的程序可读性更高，尝试为你自己的代码添加相应的注释。

---

---

## 第一节 分析模拟环境的数据

首先，只有足够了解机器人所在的环境，我们的机器人才能成功找到目标宝藏，因此首先我们来对机器人所在环境的数据进行分析。在这个部分，会考察你对数据结构、控制流的了解。


### 1.1 理解模拟环境数据的储存格式

首先我们思考这样的问题：如何存储模拟环境的数据呢？

我们将我们的模拟环境抽象成一个格子世界，每个格子按照坐标编号进行标记；每个格子中会有四个情况，分别为普通格子（可通行）、机器人的起点（可通行）、障碍物（不可通行）、宝藏箱（目标点）。例如，一个模拟环境就可以抽象成3行4列的格子世界，并按这按这样的方法进行存储：
```
environment = [[0,0,0,2], 
               [1,2,0,0],
               [0,2,3,2]]
```
我们用了一个列表来保存虚拟世界的数据。外层列表中的每一个元素依然是一个列表，它代表模拟环境中每行的数据。而对于这个列表中的每个元素都是一个数，它们的含义是：
- 0: 普通格子（可通行）
- 1: 机器人的起点（可通行）
- 2: 障碍物（不可通行）
- 3: 宝藏箱（目标点）

那么，根据上述的数据，这个迷宫的第二行第一列，是我们机器人的起点。

__注：我们描述的迷宫的坐标位置（第一行第一列），和迷宫下标索引的值（如 `(0,0)`）是不一样的，请注意下标的问题。__


如下的代码，使用了辅助函数，读取了模拟环境的数据，并保存在了 `env_data` 变量中。


In [1]:
import helper

env_data = helper.fetch_maze()

maze-id 1-1581393985
[[3, 2, 2, 2, 2, 2, 2, 2, 1],
 [0, 0, 2, 2, 2, 2, 2, 0, 0],
 [2, 0, 0, 2, 2, 2, 0, 0, 2],
 [2, 2, 0, 0, 2, 0, 0, 2, 2],
 [2, 2, 2, 0, 0, 0, 2, 2, 2]]


---


**任务1：**在如下代码中，请写代码获得这些值：

1. 模拟环境的长和宽
2. 模拟环境中第3行第6列元素

In [2]:
#TODO 1模拟环境的行数
rows = len(env_data)

#TODO 2模拟环境的列数
columns = len(env_data[0])

#TODO 3取出模拟环境第三行第六列的元素                   
row_3_col_6 = env_data[2][5] 


print("迷宫共有", rows, "行", columns, "列，第三行第六列的元素是", row_3_col_6)

迷宫共有 5 行 9 列，第三行第六列的元素是 2


---

## 1.2 分析模拟环境数据

接着我们需要对模拟环境的中的数据进行分析。请根据如下的指示，计算相应的值。

---

**任务2：**在如下代码中，请计算模拟环境中，第一行和第三列的障碍物个数。

提示：*可以用循环完成。*

In [3]:
#TODO 4计算模拟环境中，第一行的的障碍物个数。
number_of_barriers_row1 = 0
for items in env_data[0]:
    if items == 2:
        number_of_barriers_row1 = number_of_barriers_row1 + 1
        

#TODO 5计算模拟环境中，第三列的的障碍物个数。
number_of_barriers_col3 = 0
for items in range(rows):
    if env_data[items][2] == 2:
        number_of_barriers_col3 = number_of_barriers_col3 + 1
    

print("迷宫中，第一行共有", number_of_barriers_row1, "个障碍物，第三列共有", number_of_barriers_col3, "个障碍物。")

迷宫中，第一行共有 7 个障碍物，第三列共有 3 个障碍物。


In [4]:
%run -i -e test.py RobotControllortTestCase.test_cal_barriers

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


---

**任务3：**在如下代码中：

1. 创建一个名为 `loc_map` 的字典，它有两个键值，分别为 `start` 和 `destination`，对应的值分别为起点和目标点的坐标，它们以如 `(0,0)` 的形式保存为元组。
2. 从字典中取出 `start` 对应的值，保存在 `robot_current_loc` 对应的变量中，这个变量表示小车现在的位置。

In [5]:
loc_map = {'start': (0,8),'destination':(0,0)} #TODO 6按照上述要求创建字典

robot_current_loc = loc_map['start'] #TODO 7保存机器人当前的位置
#print(robot_current_loc)

In [6]:
%run -i -e test.py RobotControllortTestCase.test_cal_loc_map

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



---

---

## 第二节 控制机器人随机漫步

在这一步中，你需发出指令，控制机器人在环境中随机行动。它会考察你对控制流、调用函数的知识。



## 2.1 控制机器人行动

我们的机器人能够执行四个动作：向上走 `u`、向下走 `d`、向左走 `l`、向右走 `r`。但是，由于有障碍，很多时候机器人的行动并不能成功。所以在这里，你需要实现一个函数，来判断机器人在某个位置，执行某个移动动作是否可行。

---

**任务4：**在下方代码中，实现名为 `is_move_valid_special` 的函数，它有两个输入，分别为机器人所在的位置坐标 `loc`，以及即将执行的动作 `act`，如 `(1,1)` 及 `u`。接着它的返回是一个布尔值，表明小车在 `loc` 位置下，是否可以执行动作 `act`。


提示1：*可以读取上方定义的 `env_data` 变量，来读取模拟环境的数据。*

提示2：*在实现函数后，请删去下方的 `pass` 代码。*

提示3：*我们需要处理边界的情况，即机器人走到了虚拟环境边界时，是不能够走出虚拟环境的。*

In [7]:
def is_move_valid_special(loc, act):
    row = loc[0]
    column = loc[1]
    if act == 'd':
        if (row + 1) > len(env_data)-1:
            return False
        elif env_data[row+1][column] == 2:
            return False
        else:
            return True
        
    if act == 'u':
        if (row - 1) < 0:
            return False
        elif env_data[row-1][column] == 2:
            return False
        else:
            return True
    
    if act == 'l':
        if (column - 1) < 0:
            return False
        elif env_data[row][column-1] == 2:
            return False
        else:
            return True
    if act == 'r':
        if (column + 1) > len(env_data[0])-1:
            return False
        elif env_data[row][column+1] == 2:
            return False
        else:
            return True
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    loc -- tuple, robots current location
    act -- string, robots meant action
    """

In [8]:
%run -i -e test.py RobotControllortTestCase.test_is_move_valid_special

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


---
**任务5：**在下方代码中，重新实现一个名为 `is_move_valid` 的函数，它有三个输入，分别为模拟环境的数据 `env_data`、机器人所在的位置坐标 `loc`、以及即将执行的动作 `act`。它的返回值与此前一样，是一个布尔值，表明小车在给定的虚拟环境中的 `loc` 位置下，是否可以执行动作 `act`。

In [9]:
def is_move_valid(env_data, loc, act):
    row = loc[0]
    column = loc[1]
    if act == 'd':
        if (row + 1) > len(env_data)-1:
            return False
        elif env_data[row+1][column] == 2:
            return False
        else:
            return True
        
    if act == 'u':
        if (row - 1) < 0:
            return False
        elif env_data[row-1][column] == 2:
            return False
        else:
            return True
    
    if act == 'l':
        if (column - 1) < 0:
            return False
        elif env_data[row][column-1] == 2:
            return False
        else:
            return True
    if act == 'r':
        if (column + 1) > len(env_data[0])-1:
            return False
        elif env_data[row][column+1] == 2:
            return False
        else:
            return True
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    env -- list, the environment data
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 9
    #pass

In [10]:
%run -i -e test.py RobotControllortTestCase.test_is_move_valid

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


---

**任务6：**请回答：
  1. 在任务4及任务5中的实现的两个函数中，`env_data` 这个变量有什么不同？
  2. 调用``is_move_valid``函数，参数为``env_data_``、``loc_``、``act_``，如果在函数内修改``env_data``是否会改变``env_data_``的值？为什么？
  
提示：_可以尝试从变量作用域的角度回答该问题1。_


提示：_可以尝试从可变类型变量和不可变类型变量的角度回答该问题2。_



**回答：** （请在这里填写你的回答）
1. 
在任务4中，env_data可以适用于整个模块，同时也适用于模块下的函数，是全局变量。
在任务5中，在is_move_valid函数中的env_data这个变量只适用于函数，不会对模块里的其他变量产生影响。是局部变量。
2. 
在函数内修改env_data会改变env_data的值，因为env_data是列表类型，列表类型是可变类型变量。
列表通过“传引用”来传递对象。不同于值传递过程，在引用传递过程中，被调函数的形式参数作为局部变量虽然也在堆栈中开辟了内存空间，不过这时存放的是由主调函数放进来的实参变量的地址。被调函数对形参的任何操作都被处理成间接寻址，即通过堆栈中存放的地址访问主调函数中的实参变量。正因为如此，被调函数对形参做的任何操作都影响了主调函数中的实参变量。
答案参考网站地址：https://www.cnblogs.com/loleina/p/5276918.html

---

## 2.2 机器人可行动作

---

**任务7：**编写一个名为 `valid_actions` 的函数。它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`，输出是一个列表，表明机器人在这个位置所有的可行动作。

提示：*可以尝试调用上方定义的`is_move_valid`函数。*


In [11]:
## TODO 10 从头定义、实现你的函数
def valid_actions(env_data, loc):
    action = []
    if is_move_valid(env_data, loc, 'u'):
        action.append("u")
    if is_move_valid(env_data, loc, 'l'):
        action.append("l")
    if is_move_valid(env_data, loc, 'r'):
        action.append("r")
    if is_move_valid(env_data, loc, 'd'):
        action.append("d")
    return action

In [12]:
%run -i -e test.py RobotControllortTestCase.test_valid_actions

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


---

## 2.3 移动机器人

当机器人收到一个动作的时候，你机器人的位置应发生相应的变化。

**任务8：**编写一个名为 `move_robot` 的函数，它有两个输入，分别为机器人当前所在的位置 `loc` 和即将执行的动作 `act`。接着会返回机器人执行动作之后的新位置 `new_loc`。

In [13]:
##TODO 11 从头定义、实现你的函数
def move_robot(loc, act):
    new_loc = ()
    if act == 'u':
        new_loc = (loc[0]-1, loc[1])
    if act == 'd':
        new_loc = (loc[0]+1, loc[1])
    if act == 'r':
        new_loc = (loc[0], loc[1]+1)
    if act == 'l':
        new_loc = (loc[0], loc[1]-1)
    return new_loc

In [14]:
%run -i -e test.py RobotControllortTestCase.test_move_robot

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


---

## 2.4 随机移动机器人

接着，我们尝试在虚拟环境中随机移动机器人，看看会有什么效果。

**任务9：**编写一个名为 `random_choose_actions` 的函数，它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`。机器人会执行一个300次的循环，每次循环，他会执行以下任务：

1. 利用上方定义的 `valid_actions` 函数，找出当前位置下，机器人可行的动作；
2. 利用 `random` 库中的 `choice` 函数，从机器人可行的动作中，随机挑选出一个动作；
3. 接着根据这个动作，利用上方定义的 `move_robot` 函数，来移动机器人，并更新机器人的位置；
4. 当机器人走到终点时，输出“在第n个回合找到宝藏！”。

提示：如果机器人无法在300个回合内找到宝藏的话，试试看增大这个数字，也许会有不错的效果 :P

In [15]:
##TODO 12 从头实现你的函数
import random


def random_choose_actions(env_data, loc):
    for i in range(300):
        actions = valid_actions(env_data, loc)
        real_act = random.choice(actions)
        loc = move_robot(loc, real_act)
        if env_data[loc[0]][loc[1]] == 3:
            print('在第' + str((i+1)) + '个回合找到宝藏！')
            break
        

In [16]:
# 运行
random_choose_actions(env_data, robot_current_loc)


---

---

## （可选）第三节 控制机器人走到终点

## 3.1 控制机器人走到终点

在这里，你将综合上述的知识，编码控制机器人走到终点。这个任务对刚刚入门的你来说可能有些挑战，所以它是一个选做题。

**任务10**：尝试实现一个算法，能够对给定的模拟环境，输出机器人的行动策略，使之能够走到终点。

提示：_你可以尝试参考：_
* 深度/广度优先算法。
    以及以下参考资料：
    1. https://blog.csdn.net/raphealguo/article/details/7523411 
    2. https://www.cnblogs.com/yupeng/p/3414736.html 
* A星算法。
    以及以下参考资料：
    1. https://baike.baidu.com/item/A%2A算法 
    2. https://blog.csdn.net/hitwhylz/article/details/23089415

In [18]:
import sys
!conda install --yes --prefix {sys.prefix} point

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - point

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [22]:
##TODO 13 实现你的算法

# AStar.py
#from Point import Point


class AStar:

    class Node:
        def __init__(self, point, endPoint, g=0):
            self.point = point  # 自己的坐标
            self.father = None  # 父节点
            self.g = g     # g值
            self. h = (abs(endPoint.x - point.x)+abs(endPoint.y - point.y)) * 10  # h值的计算方法

    def __init__(self, map2d, startPoint, endPoint, passTag=0):
        """
        构造AStar算法的启动条件
        :param map2d: Array2D类型的寻路数组
        :param startPoint: Point或二元组类型的寻路起点
        :param endPoint: Point或二元组类型的寻路终点
        :param passTag: int类型的可行走标记（若地图数据!=passTag即为障碍）
        """

        # 开启表
        self.openList = []
        # 关闭表
        self.closeList = []
        # 寻路地图
        self.map2d = map2d
        # 起点终点
        if isinstance(startPoint, Point) and isinstance(endPoint, Point):
            self.startPoint = startPoint
            self.endPoint = endPoint
        else:
            self.startPoint = Point(*startPoint)
            self.endPoint = Point(*endPoint)

        # 可行走标记
        self.passTag = passTag

    def getMinNode(self):
        """
        获得openlist中F值最小的节点
        :return: Node
        """
        currentNode = self.openList[0]
        for node in self.openList:
            if node.g + node.h < currentNode.g + currentNode.h:
                currentNode = node
        return currentNode

    def pointInCloseList(self, point):
        for node in self.closeList:
            if node.point._eq_(point):
                return True
        return False

    def pointInOpenList(self, point):
        for node in self.openList:
            if node.point._eq_(point):
                return node
        return None

    def endPointInCloseList(self):
        for node in self.openList:
            if node.point._eq_(self.endPoint):
                return node
        return None

    def searchNear(self, minF, offsetX, offsetY):
        """
        搜索节点周围的点
        :param minF:F值最小的节点
        :param offsetX:坐标偏移量
        :param offsetY:
        :return:
        """
        # 越界检测
        if minF.point.x + offsetX < 0 or minF.point.x + offsetX > self.map2d.w - 1 or minF.point.y + offsetY < 0 \
                or minF.point.y + offsetY > self.map2d.h - 1:
            return

        # 如果是障碍, 就忽略
        if self.map2d.data[minF.point.x + offsetX][minF.point.y + offsetY] != self.passTag:
            return
        # 如果在关闭表中， 就忽略
        currentPoint = Point(minF.point.x + offsetX, minF.point.y + offsetY)
        # print((currentPoint.x, currentPoint.y))              # 检测代码正确性，待删除
        if self.pointInCloseList(currentPoint):
            return
        # 设置单位花费
        if offsetX == 0 or offsetY == 0:
            step = 10
        else:
            step = 14
        # 如果不在openList中，就把它加入openList
        currentNode = self.pointInOpenList(currentPoint)
        if not currentNode:
            currentNode = AStar.Node(currentPoint, self.endPoint, g=minF.g + step)
            currentNode.father = minF
            self.openList.append(currentNode)
            return
        # 如果在openList中，判断minF到当前点的G更小
        if minF.g + step < currentNode.g:   # 如果更小，就重新计算g值，并且改变father
            currentNode.g = minF.g + step
            currentNode.father = minF

    def start(self):
        """
        开始寻路
        :return: None或Point列表（路径）
        """
        #判断寻路终点是否是障碍
        if self.map2d.data[self.endPoint.x][self.endPoint.y] != self.passTag:
            return None
        # 1.将起点放入开启列表
        startNode = AStar.Node(self.startPoint, self.endPoint)
        #print(startNode)
        self.openList.append(startNode)
        #print(self.openList[0])

        # 2. 主循环逻辑
        while True:
            # 找到F值最小的点
            minF = self.getMinNode()
            # 把这个点加入closeList中，并且在openList中删除它
            # print((minF.point.x, minF.point.y))
            self.closeList.append(minF)
            # print(self.closeList)
            self.openList.remove(minF)
            # print(self.openList)

            # 判断这个节点的上下左右节点
            self.searchNear(minF, 0, -1)
            # print(self.openList)
            # print("1")                     # 找到minF上面的点
            self.searchNear(minF, 0, 1)
            # print(self.openList)
            # print("2")                      # 找到minF下面的点
            self.searchNear(minF, -1, 0)
            # print(self.openList)
            # print("3")                      # 找到minF左边的点
            self.searchNear(minF,  1, 0)
            # print(self.openList)
            # print("4")                      # 找到minF右边的点

            # 判断是否终止
            point = self.endPointInCloseList()
            if point:   # 如果终点在关闭表中， 就返回结果
                print("关闭表中")
                cPoint = point
                pathList = []
                while True:
                    if cPoint.father:
                        pathList.append(cPoint.point)
                        cPoint = cPoint.father

                    else:
                        # print(pathList)
                        # print(list(reversed(pathList)))
                        # print(pathList.reverse())
                        return list(reversed(pathList))

            if len(self.openList) == 0:
                return None
            
# Array2D.py
class Array2D:
    """
        说明：
            1.构造方法需要两个参数，即二维数组的宽和高
            2.成员变量w和h是二维数组的宽和高
            3.使用：‘对象[x][y]’可以直接取到相应的值
            4.数组的默认值都是0
    """
    def __init__(self, w, h):
        self.w = w
        self.h = h
        self.data = []
        self.data = [[0 for i in range(h)] for i in range(w)]

        """
        for y in range(self.h):
            for x in range(self.w):
                self.data[x][y] = 0
        """

    def showArray2D(self):
        for y in range(self.h):
            for x in range(self.w):
                print(self.data[x][y], end=' ')
            print("")
            
# Point.py
class Point:
    """表示一个点"""
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def _eq_(self, other):
        if self.x == other.x and self.y == other.y:
            return True
        return False

    def _str_(self):
        return "x:" + str(self.x) + ",y:" + str(self.y)
    
#demo.py
#import Array2D
#import Point
#import AStar

if __name__ == '__main__':
        # 创建一个10*10的地图
        map2d = Array2D(9, 5)

        # print(map2d.data)
        # 设置障碍
        map2d.data[1][0] = 2
        map2d.data[2][0] = 2
        map2d.data[3][0] = 2
        map2d.data[4][0] = 2
        map2d.data[5][0] = 2
        map2d.data[6][0] = 2
        map2d.data[7][0] = 2
        map2d.data[2][1] = 2
        map2d.data[3][1] = 2
        map2d.data[4][1] = 2
        map2d.data[5][1] = 2
        map2d.data[6][1] = 2
        map2d.data[0][2] = 2
        map2d.data[3][2] = 2
        map2d.data[4][2] = 2
        map2d.data[5][2] = 2
        map2d.data[8][2] = 2
        map2d.data[0][3] = 2
        map2d.data[1][3] = 2
        map2d.data[4][3] = 2
        map2d.data[7][3] = 2
        map2d.data[8][3] = 2
        map2d.data[0][4] = 2
        map2d.data[1][4] = 2
        map2d.data[2][4] = 2
        map2d.data[6][4] = 2
        map2d.data[7][4] = 2
        map2d.data[8][4] = 2

        map2d.showArray2D()

        # 创建AStar对象， 并设置起点为8, 0 终点为0, 0
        aStar = AStar(map2d, Point(8,0), Point(0,0))

        # 开始寻路
        pathList = aStar.start()

        # 将计算得到的路径用8表示
        for point in pathList:
            map2d.data[point.x][point.y] = 8
        print("----------------------")
        # 再次显示地图
        map2d.showArray2D()

0 2 2 2 2 2 2 2 0 
0 0 2 2 2 2 2 0 0 
2 0 0 2 2 2 0 0 2 
2 2 0 0 2 0 0 2 2 
2 2 2 0 0 0 2 2 2 
关闭表中
----------------------
8 2 2 2 2 2 2 2 0 
8 8 2 2 2 2 2 8 8 
2 8 8 2 2 2 8 8 2 
2 2 8 8 2 8 8 2 2 
2 2 2 8 8 8 2 2 2 


> 注意: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。